In [1]:
import pandas as pd
import numpy as np
#from sklearn.preprocessing import OneHotEncoder


anime_data = pd.read_csv('./data/anime.csv')
#print(anime_data)

user_ratings = pd.read_csv('./data/rating.csv')

genres_dummies = anime_data['genre'].str.get_dummies(sep=', ')

anime_data = pd.concat([anime_data, genres_dummies], axis=1)
#display(anime_data)

#display(anime_data.head(1).to_csv(index=False))

In [2]:
display(user_ratings)

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


In [3]:
user_ratings = user_ratings[user_ratings["rating"] != -1]
display(user_ratings)

,user_id,anime_id,rating
47,1,8074,10
81,1,11617,10
83,1,11757,10
101,1,15451,10
153,2,11771,10
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


In [4]:
user_ratings = user_ratings.dropna()

In [5]:
no_users = len(user_ratings['user_id'].unique().tolist())
no_animes = anime_data.shape[0]
print(f"Matriz de ratings {100 * user_ratings.shape[0]/ (no_users * no_animes)}% preenchida")

Matriz de ratings 0.7406239890872849% preenchida


In [6]:
no_ratings = user_ratings.shape[0]
user_ratings = user_ratings[user_ratings['user_id'] < 15000]
print(f'Qte antiga = {no_users}\nQte Nova = { len(user_ratings["user_id"].unique().tolist()) }')
print(f'Qte antiga = {no_ratings}\nQte Nova = {user_ratings.shape[0]}')

Qte antiga = 69600
Qte Nova = 14177
Qte antiga = 6337241
Qte Nova = 1261722


In [7]:
!pip install surprise

In [8]:


# Supondo que você tenha os DataFrames ratings e df
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import train_test_split

# Carregando os dados usando o Surprise
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(user_ratings[['user_id', 'anime_id', 'rating']], reader)

# Dividindo os dados em conjunto de treinamento e teste
trainset, testset = train_test_split(data, test_size=0.25)

# Usando algoritmo KNN com Surpresa (usando cosine similarity)
sim_options = {
    'name': 'cosine',
    'user_based': True
}
algo = KNNBasic(sim_options=sim_options)

# Treinando o modelo
algo.fit(trainset)

# Fazendo previsões para o conjunto de teste
predictions = algo.test(testset)

# Avaliando o desempenho do modelo (opcional)
from surprise import accuracy
accuracy.rmse(predictions)





Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.4695


1.4694561599922566

In [11]:
user_id_to_recommend = 5 #@param type: number
no_top_rated_animes = 10 #@param type: number
user_i_rated_itens = user_ratings[user_ratings['user_id'] == user_id_to_recommend]
rated_itens = user_i_rated_itens[['anime_id', 'rating']].sort_values(by='rating', ascending=False)
#display(rated_itens)
user_top_rated_itens = pd.DataFrame(columns=['name', 'predict rating'])
print(f'Top {min(no_top_rated_animes, rated_itens.shape[0])} User {user_id_to_recommend} most rated animes:')
for i in range(min(no_top_rated_animes, rated_itens.shape[0])):
  anime = anime_data[anime_data['anime_id'] == rated_itens.iloc[i]['anime_id']]
  print(f'\t#{i+1}: {anime.iloc[0]["name"]} = {rated_itens.iloc[i]["rating"]}')
  #user_top_rated_itens.append(anime.iloc[0]['name'])

#print(user_top_rated_itens)
items_to_predict = anime_data[~anime_data['anime_id'].isin(user_ratings[user_ratings['user_id'] == user_id_to_recommend]['anime_id'])]['anime_id']
for item_id in items_to_predict:
    pred = algo.predict(user_id_to_recommend, item_id)
    if pred.est > 8.99:
      user_top_rated_itens.loc[len(user_top_rated_itens)] = {'name': anime_data[anime_data['anime_id'] == item_id].iloc[0]['name'], 'predict rating': round(pred.est)}
    #print(f"Usuário {user_id_to_recommend} pode dar uma previsão de rating {pred.est:.2f} para o anime de ID {anime_data[anime_data['anime_id'] == item_id].iloc[0]['name']}")

display(user_top_rated_itens.sort_values(by='predict rating', ascending=False))

Top 10 User 5 most rated animes:
	#1: Great Teacher Onizuka = 10
	#2: Gintama Movie: Kanketsu-hen - Yorozuya yo Eien Nare = 10
	#3: Gintama&#039; = 9
	#4: Haikyuu!! Second Season = 9
	#5: Stranger: Mukou Hadan = 9
	#6: Gintama = 9
	#7: Detroit Metal City = 9
	#8: JoJo no Kimyou na Bouken (TV) = 9
	#9: Steins;Gate = 9
	#10: Mob Psycho 100 = 9


,name,predict rating
90,Ketsuekigata-kun! 4,10
132,Kinpatsu no Jeanie,10
129,Kamiusagi Rope x Panasonic Collaboration,10
33,Patalliro!,10
34,Shin Chou Kyou Ryo: Condor Hero III,10
...,...,...
46,Battle Spirits: Shounen Gekiha Dan,9
45,Mama wa Shougaku 4 Nensei,9
43,Soukyuu no Fafner: Dead Aggressor - Exodus Spe...,9
41,Doraemon Movie 32: Nobita to Kiseki no Shima -...,9


In [25]:
display(anime_data)

,anime_id,name,genre,type,episodes,rating,members,Action,Adventure,Cars,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12290,5543,Under World,Hentai,OVA,1,4.28,183,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175,0,0,0,...,0,0,0,0,0,0,0,0,0,0
